# Gerar arquivos a partir do *dataset* carregado do *CSV*

## Carregar arquivos

In [2]:
import pandas as pd

dataframe: pd.DataFrame = pd.read_csv("ibge_news_complete.csv").sample(frac=1)[["editorias", "texto"]]

dataframe

,editorias,texto
4722,NaN,Taxa representou a segunda evolução positiva c...
686,economicas,"Em agosto de 2022, a produção industrial nacio..."
2521,ibge,De 6 a 9 de novembro acontece o 7º Seminário d...
1589,sociais,"Cerca de 1,1 milhão de pessoas que estavam afa..."
2876,geociencias,O Censo Agro 2017 já está em seu terceiro mês ...
...,...,...
930,economicas,"A safra nacional de grãos deve alcançar 261,6 ..."
509,geociencias,O IBGE realizou mais uma atualização na lista ...
4367,NaN,O Índice Nacional de Preços ao Consumidor Ampl...
4130,NaN,"Em 2011, a esperança de vida ao nascer no Bras..."


Remove NaNs

In [3]:
dataframe = dataframe.dropna()

## Generate files in format of Segformer

Função para tokenizar sentenças nos textos

In [4]:
import nltk

def tokenize_sentences(text: str) -> str:
    return "\n".join(nltk.sent_tokenize(text=str(text)))

Tokenizar sentenças dos documentos

In [5]:
dataframe["texto"] = dataframe["texto"].apply(tokenize_sentences).apply(str)

dataframe

/tmp/ipykernel_48976/2328746594.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["texto"] = dataframe["texto"].apply(tokenize_sentences).apply(str)


,editorias,texto
686,economicas,"Em agosto de 2022, a produção industrial nacio..."
2521,ibge,De 6 a 9 de novembro acontece o 7º Seminário d...
1589,sociais,"Cerca de 1,1 milhão de pessoas que estavam afa..."
2876,geociencias,O Censo Agro 2017 já está em seu terceiro mês ...
3200,economicas,O IBGE realizou mais uma atualização semestral...
...,...,...
2026,economicas,Os super e hipermercados impediram que as vend...
345,economicas,A estimativa de julho para a safra nacional de...
930,economicas,"A safra nacional de grãos deve alcançar 261,6 ..."
509,geociencias,O IBGE realizou mais uma atualização na lista ...


Dividir o *dataset* em conjuntos de treino (70%), teste (20%) e validação (10%)

In [17]:
import numpy as np

train_df, test_df, val_df = np.split(dataframe, [int(0.7 * len(dataframe)), int(0.9 * len(dataframe))])
train_df


/home/wsl/.pyenv/versions/3.11.6/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,editorias,texto
686,economicas,"Em agosto de 2022, a produção industrial nacio..."
2521,ibge,De 6 a 9 de novembro acontece o 7º Seminário d...
1589,sociais,"Cerca de 1,1 milhão de pessoas que estavam afa..."
2876,geociencias,O Censo Agro 2017 já está em seu terceiro mês ...
3200,economicas,O IBGE realizou mais uma atualização semestral...
...,...,...
2191,economicas,A Pesquisa Industrial Anual (PIA) Empresa most...
1127,ibge,A Escola Nacional de Ciências Estatísticas (EN...
2361,economicas,"A prévia da inflação ficou em 0,30% em janeiro..."
2962,economicas,"Em relação à produção de agosto (240,9 milhões..."


Salvar os conjuntos em arquivo.

In [22]:
train_df.to_csv("train.csv")
test_df.to_csv("test.csv")
val_df.to_csv("val.csv")

In [20]:
val_df

,editorias,texto
3661,ibge,"Em fevereiro, o comércio varejista do país reg..."
2113,geociencias,"Mais de um quarto (26,2%) dos desempregados pr..."
3927,geociencias,O Índice Nacional da Construção Civil (Sinapi)...
269,economicas,O Índice Nacional da Construção Civil (Sinapi)...
2363,revistaretratos,A Escola Nacional de Ciências Estatísticas (EN...
...,...,...
2026,economicas,Os super e hipermercados impediram que as vend...
345,economicas,A estimativa de julho para a safra nacional de...
930,economicas,"A safra nacional de grãos deve alcançar 261,6 ..."
509,geociencias,O IBGE realizou mais uma atualização na lista ...


Caso não queira aleatorizar o tamanho dos arquivos, importe a lista com os seus tamanhos de um arquivo salvo. Caso contrário, não execute esta célula.

In [ ]:
import json

number_docs_series_dict_train: dict = {}
number_docs_series_dict_test: dict = {}
number_docs_series_dict_validation: dict = {}

with open("number_docs_train.json") as f_in:
    number_docs_series_dict_train = json.load(f_in)
    
with open("number_docs_test.json") as f_in:
    number_docs_series_dict_test = json.load(f_in)
    
with open("number_docs_val.json") as f_in:
    number_docs_series_dict_test = json.load(f_in)
    
number_docs_series_dict_train

Estatística dos conjuntos (trocar conjunto ao executar a célula)

In [21]:
news_categories: list[str] = dataframe["editorias"].unique()

[{ "category": cat, "abs_frequency": list(val_df["editorias"]).count(cat), "rel_frequency": list(val_df["editorias"]).count(cat) / len(val_df)} for cat in news_categories]

[{'category': 'economicas',
  'abs_frequency': 227,
  'rel_frequency': 0.6005291005291006},
 {'category': 'ibge',
  'abs_frequency': 48,
  'rel_frequency': 0.12698412698412698},
 {'category': 'sociais',
  'abs_frequency': 51,
  'rel_frequency': 0.1349206349206349},
 {'category': 'geociencias',
  'abs_frequency': 24,
  'rel_frequency': 0.06349206349206349},
 {'category': 'ibge;censo2020',
  'abs_frequency': 9,
  'rel_frequency': 0.023809523809523808},
 {'category': 'sociais;geociencias', 'abs_frequency': 0, 'rel_frequency': 0.0},
 {'category': 'seriesespeciais',
  'abs_frequency': 11,
  'rel_frequency': 0.0291005291005291},
 {'category': 'revistaretratos',
  'abs_frequency': 5,
  'rel_frequency': 0.013227513227513227},
 {'category': 'sociais;economicas',
  'abs_frequency': 2,
  'rel_frequency': 0.005291005291005291},
 {'category': 'censo2020',
  'abs_frequency': 1,
  'rel_frequency': 0.0026455026455026454},
 {'category': 'seriesespeciais;censo2020',
  'abs_frequency': 0,
  'rel_frequenc

Grava *datasets* no formato do Segformer

In [23]:
import os, sys

sys.path.append("../utils")

from convert_segformer import generate_segformer_docs

merged_texts_df_train: pd.DataFrame = generate_segformer_docs(
    dataframe=train_df, 
    folder="train", 
    text_body_field="texto", 
    max_doc_size=4, 
    category_field="editorias"
)

merged_texts_df_train

,fulltext,number_sentences,number_words,number_docs,average_words_per_sentence,median_words_per_sentence
0,"Em agosto de 2022, a produção industrial nacio...",101,4242,4,42.000000,35.0
1,O IBGE realizou mais uma atualização semestral...,10,420,1,42.000000,29.5
2,"A taxa recorde de desocupação de 14,7% no país...",51,1609,1,31.549020,27.0
3,O Índice Nacional da Construção Civil (Sinapi)...,10,422,1,42.200000,39.0
4,O Índice Nacional de Preços ao Consumidor Ampl...,118,5079,4,43.042373,34.0
...,...,...,...,...,...,...
1057,O setor de serviços apresentou queda de 4% na ...,53,2907,3,54.849057,52.0
1058,"Em maio, o Índice Nacional de Preços ao Consum...",91,5349,4,58.780220,49.0
1059,A Escola Nacional de Ciências Estatísticas (EN...,15,580,1,38.666667,31.0
1060,"A prévia da inflação ficou em 0,30% em janeiro...",49,2308,2,47.102041,39.0


In [24]:

merged_texts_df_test: pd.DataFrame = generate_segformer_docs(
    dataframe=test_df, 
    folder="test", 
    text_body_field="texto", 
    max_doc_size=4, 
    category_field="editorias"
)

merged_texts_df_test

,fulltext,number_sentences,number_words,number_docs,average_words_per_sentence,median_words_per_sentence
0,"Cerca de 82,3% dos jovens de 15 a 29 anos que ...",24,959,1,39.958333,37.5
1,"No mês de julho, o setor de serviços cresceu 0...",41,2049,2,49.975610,44.0
2,"A safra nacional de cereais, leguminosas e ole...",22,945,2,42.954545,40.0
3,O IBGE divulgou hoje (07) as Contas Econômicas...,55,2749,3,49.981818,37.0
4,Os resultados da produção animal no 1º trimest...,26,1677,1,64.500000,49.5
...,...,...,...,...,...,...
294,O total de empresas em atividade no país era d...,36,1794,2,49.833333,31.0
295,"A inflação oficial do mês de maio ficou em 0,2...",59,2940,2,49.830508,47.0
296,"Em fevereiro de 2023, o volume de vendas no co...",99,5664,4,57.212121,52.0
297,O IBGE divulga amanhã (05/11) o Índice de Preç...,4,146,1,36.500000,37.5


In [25]:
merged_texts_df_val: pd.DataFrame = generate_segformer_docs(
    dataframe=val_df, 
    folder="val", 
    text_body_field="texto", 
    max_doc_size=4, 
    category_field="editorias"
)

merged_texts_df_val

,fulltext,number_sentences,number_words,number_docs,average_words_per_sentence,median_words_per_sentence
0,"Em fevereiro, o comércio varejista do país reg...",35,2325,2,66.428571,50.0
1,O Índice Nacional da Construção Civil (Sinapi)...,20,1021,2,51.050000,48.5
2,A Escola Nacional de Ciências Estatísticas (EN...,38,1595,4,41.973684,38.0
3,"Em janeiro, o volume do setor de serviços teve...",26,921,1,35.423077,34.0
4,"O volume de serviços no Brasil recuou 1,0% em ...",27,1548,1,57.333333,46.0
...,...,...,...,...,...,...
151,O Índice de Preços ao Consumidor Amplo-15 (IPC...,26,1200,1,46.153846,42.0
152,"O IBGE, o Ministério da Saúde e a Universidade...",12,637,2,53.083333,47.5
153,As vendas no comércio varejista no país recuar...,36,2017,3,56.027778,44.5
154,Os super e hipermercados impediram que as vend...,28,1399,2,49.964286,45.5


Conta tamanhos de documentos (medidos em notícias) para cada um dos conjuntos.

In [30]:
list(merged_texts_df_train["number_docs"]).count(4)


255

In [34]:
list(merged_texts_df_test["number_docs"]).count(4)

68

In [38]:
list(merged_texts_df_val["number_docs"]).count(4)

40

Contagem de palavras, frases e documentos por conjunto.

In [40]:
print("Documentos gerados: ", len(merged_texts_df_train))
print("Palavras: ", merged_texts_df_train["number_words"].sum())
print("Sentenças: ", merged_texts_df_train["number_sentences"].sum())

Documentos gerados:  1062
Palavras:  3047504
Sentenças:  64881


In [41]:
print("Documentos gerados: ", len(merged_texts_df_test))
print("Palavras: ", merged_texts_df_test["number_words"].sum())
print("Sentenças: ", merged_texts_df_test["number_sentences"].sum())

Documentos gerados:  299
Palavras:  889779
Sentenças:  17918


In [42]:
print("Documentos gerados: ", len(merged_texts_df_val))
print("Palavras: ", merged_texts_df_val["number_words"].sum())
print("Sentenças: ", merged_texts_df_val["number_sentences"].sum())

Documentos gerados:  156
Palavras:  435643
Sentenças:  9021


Salvar série de tamanhos de notícias em arquivo.

In [44]:
import json

with open("number_docs_train.json", 'w') as fp:
    json.dump({ "number_docs": list(merged_texts_df_train["number_docs"])}, fp)
with open("number_docs_test.json", 'w') as fp1:
    json.dump({ "number_docs": list(merged_texts_df_test["number_docs"])}, fp1)
with open("number_docs_val.json", 'w') as fp2:
    json.dump({ "number_docs": list(merged_texts_df_val["number_docs"])}, fp2)